## ORIGINAL DATA

In [166]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
csv_path = "Resources/04-Pandas_homework_Instructions_HeroesOfPymoli_Resources_purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
file_df = pd.read_csv(csv_path)
#sort by SN alphabetically
file_df = file_df.sort_values("SN")

file_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
467,467,Adairialis76,16,Male,123,Twilight's Carver,2.28
142,142,Adastirin33,35,Female,175,Woeful Adamantite Claymore,4.48
388,388,Aeda94,17,Male,128,"Blazeguard, Reach of Eternity",4.91
28,28,Aela59,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",4.32
630,630,Aelaria33,23,Male,171,Scalpel,1.79


## Player Count

In [167]:
#PLAYER COUNT
print("PLAYER COUNT")
unique_players = file_df["SN"].nunique()
print(f"Total Unique Players: {unique_players}")
print(f"Total Plays: {file_df['SN'].count()}")

PLAYER COUNT
Total Unique Players: 576
Total Plays: 780


## Purchasing Analysis (Total)

* Number of Unique Items
* Average Purchase Price
* Total Number of Purchases
* Total Revenue

In [168]:
#Extract the necessary data from the main file
PA_df = file_df[['Purchase ID','Item Name', 'Price']]
##PA_df.head()

#NUMBER OF UNIQUE ITEMS + PURCHASES
unique_items = PA_df["Item Name"].nunique()
print(f"Total Unique Items: {unique_items}")

unique_purchases = PA_df["Purchase ID"].nunique()
print(f"Total Unique Purchases: {unique_purchases}")

#TOTAL REVENUE
total_rev = file_df["Price"].sum()
print(f"Total Revenue: ${total_rev}")

#Grouped by Item Name
item_group = PA_df.groupby(["Item Name"])
#Found the Avg of each Item
average_purchase_price = item_group.mean()
#Format Price to 2 decimals w/ a $ sign
average_purchase_price["Price"] = average_purchase_price["Price"].astype(float).map("${:.2f}".format)
#Rename Price to Avg Price
average_purchase_price = average_purchase_price.rename(columns={"Price":"Avg. Price"})
#Drop the Purchase ID
average_purchase_price = average_purchase_price.drop(columns=["Purchase ID"])

average_purchase_price.head()

Total Unique Items: 179
Total Unique Purchases: 780
Total Revenue: $2379.77


,Avg. Price
Item Name,
Abyssal Shard,$2.67
"Aetherius, Boon of the Blessed",$3.39
Agatha,$3.08
Alpha,$2.07
"Alpha, Oath of Zeal",$4.05


In [169]:
#TOTAL AVG
total_avg = file_df["Price"].mean()
total_avg = round(total_avg,2)
print(f"Avg Item Price: ${total_avg}")

Avg Item Price: $3.05


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [170]:
#CREATE THE DF
#Remove duplicates of the SN
de_duped_sns = file_df.drop_duplicates(subset="SN")

gender_group = de_duped_sns.drop(columns=["Purchase ID","Age","Item ID","Item Name","Price"])
gender_group = gender_group.rename(columns={"SN":"Gender Count"})

#Group by Gender
gender_group = gender_group.groupby(["Gender"])

#Convert Group to DF
gender_df = pd.DataFrame(gender_group["Gender Count"].count())
#gender_df.head()

In [171]:
#DETERMINE THE PERCENTAGES AND MAKE THEM INTO A SEPARATE DF
female_percent = gender_df.loc["Female","Gender Count"]/unique_players*100
#print(f"Female Players: {female_percent}%")

male_percent = gender_df.loc["Male","Gender Count"]/unique_players*100
#print(f"Male Players: {male_percent}%")

non_disclosed_percent = gender_df.loc["Other / Non-Disclosed","Gender Count"]/unique_players*100
#print(f"Non-Disclosed Players: {non_disclosed_percent}%")

gen_percent_df = pd.DataFrame({
    "Gender": ["Female","Male","Other / Non-Disclosed"],
    "Gender Percent": [female_percent, male_percent, non_disclosed_percent]})
#gen_percent_df

In [172]:
#MERGE THE TWO DFs

merge_genders = pd.merge(gender_df, gen_percent_df, on="Gender")
#set new index
merge_genders = merge_genders.set_index("Gender")
#format percents
merge_genders["Gender Percent"] = merge_genders["Gender Percent"].astype(float).map("{:.2f}%".format)

merge_genders 

,Gender Count,Gender Percent
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender

The below each broken by gender

* Purchase Count
* Average Purchase Price
* Total Purchase Value
* Average Purchase Total per Person by Gender


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [173]:
#Create thr group object
gender_purchase_group = file_df.groupby(["Gender"])
#gender_purchase_group 

In [174]:
#PURCHASE COUNT
#Convert Group to DF
purchases_count_df = pd.DataFrame(gender_purchase_group["Purchase ID"].count())
#Rename
purchases_count_df = purchases_count_df.rename(columns={"Purchase ID":"Purchase Count"})

#gender_purchases_df.head()

In [175]:
#AVG PURCHASE PRICE
#Convert Group to DF
purchases_price_df = pd.DataFrame(gender_purchase_group["Price"].mean())
#Rename
purchases_price_df = purchases_price_df.rename(columns={"Price":"Avg Purchase Price"})

#format
purchases_price_df = purchases_price_df["Avg Purchase Price"].astype(float).map("${:.2f}".format)

#purchases_price_df.head()

In [176]:
#TOTAL PURCHASE VALUE
#Convert Group to DF
purchases_total_df = pd.DataFrame(gender_purchase_group["Price"].sum())
#Rename
purchases_total_df = purchases_total_df.rename(columns={"Price":"Total Purchases Cost"})

#format
purchases_total_df = purchases_total_df["Total Purchases Cost"].astype(float).map("${:.2f}".format)

#purchases_total_df.head()

In [202]:
#MERGE THE THREE DFs

triple_merge = purchases_count_df.merge(purchases_price_df, how='outer', left_index=True, right_index=True)
triple_merge = triple_merge.merge(purchases_total_df, how='outer', left_index=True, right_index=True)

triple_merge.head()


,Purchase Count,Avg Purchase Price,Total Purchases Cost
Gender,,,
Female,113,$3.20,$361.94
Male,652,$3.02,$1967.64
Other / Non-Disclosed,15,$3.35,$50.19


In [178]:
#Average Purchase Total per Person by Gender
gender_purchase_group2 = file_df.groupby(["Gender","SN"])
#gender_purchase_group2

#AVG PURCHASE PRICE
#Convert Group to DF
purchases_price_df2 = pd.DataFrame(gender_purchase_group2["Price"].mean())
#Rename
purchases_price_df2 = purchases_price_df2.rename(columns={"Price":"Avg Purchase Price ($)"})

#purchases_price_df.head()
purchases_price_df2.head()

Avg Purchase Price ($)
Gender SN                                   
Female Adastirin33                      4.48
       Aerithllora36                    4.32
       Aethedru70                       3.54
       Aidain51                         3.45
       Aiduesu86                        4.48

## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [179]:
max_age = de_duped_sns["Age"].max()
#print(max_age)

min_age = de_duped_sns["Age"].min()
#print(min_age)

In [180]:
#Create the Bins
bins = [0,10,19,29,39,49]
label_names = ["Kids","Teens","20's","30's","40+"]

In [181]:
# Slice the data and place it into bins
de_duped_sns["Age Group"] = pd.cut(de_duped_sns["Age"],bins,labels=label_names)
de_duped_sns.head()

/Users/Dodge/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Group
467,467,Adairialis76,16,Male,123,Twilight's Carver,2.28,Teens
142,142,Adastirin33,35,Female,175,Woeful Adamantite Claymore,4.48,30's
388,388,Aeda94,17,Male,128,"Blazeguard, Reach of Eternity",4.91,Teens
28,28,Aela59,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",4.32,20's
630,630,Aelaria33,23,Male,171,Scalpel,1.79,20's


In [182]:
#Create thr group object
age_group = de_duped_sns.groupby(["Age Group"])

#Convert Group to DF
age_group_df = pd.DataFrame(age_group["Age Group"].count())
#Rename
age_group_df = age_group_df.rename(columns={"Age Group":"Count"})

age_group_df.head()

,Count
Age Group,
Kids,24
Teens,122
20's,335
30's,83
40+,12


In [183]:
#DETERMINE THE PERCENTAGES...
kids_percent = age_group_df.loc["Kids","Count"]/unique_players*100
#print(kids_percent)

teens_percent = age_group_df.loc["Teens","Count"]/unique_players*100
#print(teens_percent)

twentys_percent = age_group_df.loc["20's","Count"]/unique_players*100
#print(twentys_percent)

thirtys_percent = age_group_df.loc["30's","Count"]/unique_players*100
#print(thirtys_percent)

fortyplus_percent = age_group_df.loc["40+","Count"]/unique_players*100
#print(fortyplus_percent)

#...AND MAKE THEM INTO A SEPARATE DF...

age_precents_df = pd.DataFrame({
    "Age Group": ["Kids","Teens","20's","30's","40+"],
    "Age Percent": [kids_percent, teens_percent, twentys_percent,
                   thirtys_percent,fortyplus_percent]})
#...MERGE THEM TOGETHER...
merge_ages = pd.merge(age_group_df, age_precents_df, on="Age Group")

#format percents
merge_ages["Age Percent"] = merge_ages["Age Percent"].astype(float).map("{:.2f}%".format)
print("AGE DEMOGRAPHICS TABLE")
merge_ages

AGE DEMOGRAPHICS TABLE


,Age Group,Count,Age Percent
0,Kids,24,4.17%
1,Teens,122,21.18%
2,20's,335,58.16%
3,30's,83,14.41%
4,40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [204]:
#Purchase Count
pc_df = pd.DataFrame(age_group["Price"].count())
#Rename
pc_df = pc_df.rename(columns={"Price":"Count"})
#pc_df

In [205]:
#Avg Purchase Price
age_analysis_df = pd.DataFrame(age_group["Price"].mean())
#Rename
age_analysis_df = age_analysis_df.rename(columns={"Price":"Avg Price"})
age_analysis_df["Avg Price"] = age_analysis_df["Avg Price"].astype(float).map("${:.2f}".format)
#age_analysis_df 

In [206]:
#Total Purchase Value
tpv_df = pd.DataFrame(age_group["Price"].sum())
#Rename
tpv_df = tpv_df.rename(columns={"Price":"Total Purchase Value"})
tpv_df["Total Purchase Value"] = tpv_df["Total Purchase Value"].astype(float).map("${:.2f}".format)
#tpv_df

In [203]:
#Triple Merge Summary
purchase_analysis_summary = pc_df.merge(age_analysis_df, how='outer', left_index=True, right_index=True)
purchase_analysis_summary = purchase_analysis_summary.merge(tpv_df, how='outer', left_index=True, right_index=True)

purchase_analysis_summary.head()

,Count,Avg Price,Total Purchase Value
Age Group,,,
Kids,24,$3.43,$82.28
Teens,122,$3.00,$365.73
20's,335,$3.01,$1008.86
30's,83,$3.19,$265.14
40+,12,$2.90,$34.84


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [231]:
top_spenders = file_df.groupby("SN")
#print(top_spenders)

In [229]:
#Purchase Count
TS_purchasecount = pd.DataFrame(top_spenders["Purchase ID"].count())
TS_purchasecount  = TS_purchasecount .rename(columns={"Purchase ID":"Purchase Count"})
#TS_purchasecount.head()

In [230]:
#Average Purchase Value
TS_AVGpurchasevalue = pd.DataFrame(top_spenders["Price"].mean())
TS_AVGpurchasevalue  = TS_AVGpurchasevalue.rename(columns={"Price":"Avg Purchase Value"})
#TS_AVGpurchasevalue.head()

In [232]:
#Total Purchase Value
TS_TTLpurchasevalue = pd.DataFrame(top_spenders["Price"].sum())
TS_TTLpurchasevalue = TS_TTLpurchasevalue.rename(columns={"Price":"Total Purchase Value"})
#TS_TTLpurchasevalue.head()

In [226]:
#Merge
merge_TopSpenders = pd.merge(TS_TTLpurchasevalue, TS_purchasecount, on="SN")
merge_TopSpenders = pd.merge(merge_TopSpenders, TS_AVGpurchasevalue, on="SN")
#Sort
sortby_PV = merge_TopSpenders.sort_values("Total Purchase Value", ascending = False)
#Format
sortby_PV["Total Purchase Value"] = sortby_PV["Total Purchase Value"].astype(float).map("${:.2f}".format)
sortby_PV["Avg Purchase Value"] = sortby_PV["Avg Purchase Value"].astype(float).map("${:.2f}".format)

sortby_PV.head()

,Total Purchase Value,Purchase Count,Avg Purchase Value
SN,,,
Lisosia93,$18.96,5,$3.79
Idastidru52,$15.45,4,$3.86
Chamjask73,$13.83,3,$4.61
Iral74,$13.62,4,$3.41
Iskadarya95,$13.10,3,$4.37


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [264]:
#extract the data
most_pop_df = file_df[["Item ID","Item Name","Price"]]
#Remove duplicates of the SN
most_pop_df = most_pop_df.drop_duplicates(subset="Item Name")
#most_pop_df.head()

In [265]:
#group it
most_pop_group = file_df.groupby(["Item ID","Item Name"])

#print(most_pop_group)

In [266]:
#Purchase Count
MP_purchasecount = pd.DataFrame(most_pop_group["Purchase ID"].count())
MP_purchasecount  = MP_purchasecount .rename(columns={"Purchase ID":"Purchase Count"})
#MP_purchasecount.head()

In [267]:
#Total Purchase Value
MP_tpv = pd.DataFrame(most_pop_group["Price"].sum())
MP_tpv  = MP_tpv .rename(columns={"Price":"Total Purchase Value"})
#MP_tpv .head()

In [274]:
#Merge
merge_most_pop = pd.merge(most_pop_df, MP_purchasecount, on="Item Name")
merge_most_pop = pd.merge(merge_most_pop, MP_tpv, on="Item Name")

#set new index
merge_most_pop = merge_most_pop.set_index("Item ID")

#Sort
merge_most_pop = merge_most_pop.sort_values("Total Purchase Value", ascending = False)
#Format
merge_most_pop["Total Purchase Value"] = merge_most_pop["Total Purchase Value"].astype(float).map("${:.2f}".format)
merge_most_pop["Price"] = merge_most_pop["Price"].astype(float).map("${:.2f}".format)

## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [275]:
merge_most_pop.head()

,Item Name,Price,Purchase Count,Total Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,12,$50.76
82,Nirvana,$4.90,9,$44.10
145,Fiery Glass Crusader,$4.58,9,$41.22
92,Final Critic,$4.88,8,$39.04
92,Final Critic,$4.88,5,$39.04
